Before running this application, please remove and replace all files under data/inventory and data/updates in order to execute latest changes.

data/inventory
https://admin.shoplineapp.com/admin/waddystore/reports?createdBy=pos
實體店 > 庫存總表
Retail Store > Product Inventory Report

data/updates
https://admin.shoplineapp.com/admin/waddystore/products/import/bulkupload_v2
匯出商品更新檔
Export Product Bulk Update File

In [1]:
import inv_dataset
import browser
import os
import pandas as pd
import csv

All Completed, End Task.


In [2]:
inventory_folder = 'data/inventory/'
udpate_folder_path = 'data/updates/'

#convert inventory xls to csv
inventory_files = inv_dataset.Converter()
inventory_files.to_csv(inventory_folder)

#convert updates xls to csv
update_files = inv_dataset.Converter()
update_files.to_csv(udpate_folder_path)

In [3]:
#read inventory csv into dataframe
inventory = inv_dataset.Reader()
inv_df = inventory.get_df(inventory_folder)

In [4]:
'''
back to parent directory if necessery
Use this line: 
os.chdir("../..")

check current directory:
os.getcwd()
'''

os.getcwd()

'c:\\Users\\Waddy Store\\Documents\\Code Project\\shopline_automation'

In [5]:
def PreOrderClose():
    process_list = []
    #Start update procedure
    os.chdir(udpate_folder_path)
    #go thru each bulk update form csv
    for file in os.listdir():
        if file.endswith(".csv"):
            #in each bulk update form csv
            file_path = f"{file}"
            df = pd.read_csv(file_path, index_col=None, header=0)
            df = df.drop(index=0)

            df = df.loc[df['Preorder Product'] == "Y"]
            df = df.loc[df['Status'] == "Y"]
            
            for barcode in df['Barcode']:
                df2 = inv_df.loc[inv_df['商品條碼'] == barcode]
                df2 = df2.loc[df2['預設倉庫'] > 0]
                for bar in df2['商品條碼']:
                    sku_id = df[df['Barcode']== bar]['Product ID (DO NOT EDIT)'].item()
                    varient_id = df[df['Barcode']== bar]['Variant ID (DO NOT EDIT)'].item()
                    has_varient = True
                    if pd.isnull(varient_id):
                        has_varient = False
                    process_list.append([sku_id, bar, has_varient])

    os.chdir("../..") 
    print(process_list) 
    clicker = browser.Action()
    clicker.close_preorder(process_list)   
    print("All Completed, End Task.")    

In [6]:
PreOrderClose()

[['5f3b50bc09ec050048c12f99', '6971668340378', True], ['5f3c8c4da7a677003921361e', '4710006290287', False], ['5f3c90916173d6003357ca25', '4710006290218', False], ['5f3c9df00d6366003cfa30f3', '4710006290188', False], ['5fa92235e075fb16c52c0271', '69356464', False], ['60f914af9d99680029d006f8', '4580279019829', False], ['6156cc7e3c09480014c7e9dc', '4573423119346', False], ['61c43df7e560bc0026fb716e', '4582593592293', False], ['62317658eda8c40014c5a641', '4582593592309', False]]
Clicker starts
Login sucessful
Now browsing to SKU: 5f3b50bc09ec050048c12f99
Now browsing to SKU: 5f3c8c4da7a677003921361e
Now browsing to SKU: 5f3c90916173d6003357ca25
Now browsing to SKU: 5f3c9df00d6366003cfa30f3
Now browsing to SKU: 5fa92235e075fb16c52c0271
Now browsing to SKU: 60f914af9d99680029d006f8
Now browsing to SKU: 6156cc7e3c09480014c7e9dc
Now browsing to SKU: 61c43df7e560bc0026fb716e
Now browsing to SKU: 62317658eda8c40014c5a641
Go to Price and Quantity Tab
Checkbox Status is  None
pass clicking checkb

In [7]:
def PreOrderOpen():
    file = open("search/namelist.csv", "r")
    data = list(csv.reader(file, delimiter=","))
    file.close()
    search_for = [row[0] for row in data]
    process_list = []

    #Start update procedure
    #go thru each bulk update form csv
    os.chdir(udpate_folder_path)
    for file in os.listdir():
        if file.endswith(".csv"):
            #in each bulk update form csv
            file_path = f"{file}"
            update_df = pd.read_csv(file_path, index_col=None, header=0)
            update_df = update_df.drop(index=0)

            df = update_df.loc[update_df['Product Name (Traditional Chinese)'].str.contains('|'.join(search_for), na = False)]

            ''' end conidtion'''

            df = df.loc[df['Preorder Product'] == "N"]
            df = df.loc[df['Status'] == "Y"]

            discon_tag = 'dis'
            df = df.loc[~df['Product Tag'].str.contains(discon_tag, na=False)]

            discon_tag_manual = 'shortage'
            df = df.loc[~df['Product Tag'].str.contains(discon_tag_manual, na=False)]
            
            for barcode in df['Barcode']:
                df2 = inv_df.loc[inv_df['商品條碼'] == barcode]
                df2 = df2.loc[df2['預設倉庫'] <= 0]
                for bar in df2['商品條碼']:
                    sku_id = df[df['Barcode']== bar]['Product ID (DO NOT EDIT)'].item()
                    varient_id = df[df['Barcode']== bar]['Variant ID (DO NOT EDIT)'].item()
                    has_varient = True
                    if pd.isnull(varient_id):
                        has_varient = False
                    process_list.append([sku_id, bar, has_varient])

    print(process_list) 
    clicker = browser.Action()
    clicker.open_preorder(process_list)   
    print("All Completed, End Task.")    

In [8]:
PreOrderOpen()

[['5ffd80f0a167f4002fe258c1', '4560220557426', False], ['6062fa4eec7042002f6a36f0', '7350075028182', True], ['60828de37dad10003597a1cc', '4712933173367', False], ['61738c0b9525970020baaedb', '4573463893589', False], ['61a8a973e930820011531204', '71264698', False], ['61bc37facfa273002f5b418b', '4573463893541', True], ['61bc3fffcfa273002f5b44c9', '4573463893572', False], ['61c5906ae2e428236ce4e2ff', '56350922', True], ['61c5918276714800325b71b4', '70096502', True], ['61c593bb566ae40020c1ec77', '96164256', True], ['61d9cd2a54b34800262fd5f3', '36857937', True], ['62796e5849c2d900156bb270', '4571227058809', True], ['62a1a97a97f90e002724c545', '26271435', False], ['62ac4036aff40f001838ebd7', '37294609', False], ['62b324842db772002e965d4a', '53579804', False], ['62c53b923c16470032d33f11', '25512889', False], ['62c55033b0c40c0634d18783', '36911272', False], ['62c7bb2d59ceeb0023f1d483', '4573463893534', False], ['62cd318ccfd84a00297ff1e6', '36231430', True], ['62cd3812c9c55000326dc07b', '760845

TimeoutException: Message: 
